#### DNN 기반 회귀 모델 구현 + 학습 진행 모니터링 + 진행 파라미터 저장
- 데이터셋: iris.csv
- 피쳐/속성: 3개
- 타겟/라벨: 1개 Petal_width
- 학습방법: 지도학습, 회귀 
- 알고리즘: 인공신경망(ANN) -> 심층신경망(DNN, 은닉층 많음), MLP(multi-layer perceptron)
- 프레임워크: 파이토치
---
- 모니터링
    - 기준 설정: 검증데이터셋의 loss와 score
    - 평가: 학습 데이터셋의 loss와 score 비교해서 학습 중단여부 결정
    - 선택: 현재까지 진행된 모델의 파라미터(가중치, 절편) 저장 여부 또는 모델 전체 저장 여부

[1] 모듈로딩 & 데이터 준비 <hr>

In [20]:
# 모듈 로딩
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import torch.optim as optim
from torchmetrics.regression import R2Score, MeanSquaredError
from torchinfo import summary

# 데이터 처리
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import *

In [21]:
# 패키지 버전 체크
print(f' {torch.__version__}')
print(f' {pd.__version__}')

 2.4.1
 2.0.3


In [22]:
DATAFILE= '../data/iris.csv'
irisDF=pd.read_csv(DATAFILE, usecols=(0,1,2,3))
irisDF.head(3)

,sepal.length,sepal.width,petal.length,petal.width
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2


[2] 모델 클래스 설계 및 정의 <hr>
- 클래스 목적: iris 데이터 학습 및 추론
- 클래스 이름: irisReg
- 부모 클래스: nn.Module
- 매개변수: 필요없음(입출력 개수 고정)
- 속성/필드: featureDF, targetDF, n_rows, n_features
- 기능/역할: __init__() =>모델 구조 설정, forward() => 순방향 학습// 오버라이딩
- 클래스 구조
    - 입력층: 입력 3개(피쳐) 출력 10개
    - 은닉층: 입력 10개 출력 30개
    - 출력층: 입력 30개 출력 1개

- 손실함수/활성화함수
    - 클래스 형태 ==> nn.ReLU, nn.MSELoss => init()메서드
    - 함수 형태 ==> torch.nn.funtional 아래에 forward() 메서드

In [23]:
class IrisRegModel(nn.Module):
    
    # 모델 구조 구성 및 인스턴스 생성 메서드
    def __init__(self) -> None:
        super().__init__()
        self.in_layer= nn.Linear(3, 10)
        self.hidden_layer= nn.Linear(10, 30)
        self.out_layer= nn.Linear(30, 1)

    # 순방향 학습 진행 메서드
    def forward(self, input_data):
        # 입력층
        y= self.in_layer(input_data)    #변수가 3개 절편이 하나인 식이 퍼셉트론당 1개씩 나옴
        y= F.relu(y)                   #0 ~ y 사이의 값 도출

        # 은닉층
        y= self.hidden_layer(y)         #10개 입력 => 30개 출력
        y= F.relu(y)                   # 0~ y 사이의 값

        # 출력층
        y= self.out_layer(y)            #30개 입력 => 1개 출력 (petal_width 최종 예측값)
        return y

In [24]:
#  모델 인스턴스 생성
model= IrisRegModel()
print(model)

IrisRegModel(
  (in_layer): Linear(in_features=3, out_features=10, bias=True)
  (hidden_layer): Linear(in_features=10, out_features=30, bias=True)
  (out_layer): Linear(in_features=30, out_features=1, bias=True)
)


In [25]:
# 모델 사용 메모리 정보 확인
summary(model, input_size=(10, 3))

Layer (type:depth-idx)                   Output Shape              Param #
IrisRegModel                             [10, 1]                   --
├─Linear: 1-1                            [10, 10]                  40
├─Linear: 1-2                            [10, 30]                  330
├─Linear: 1-3                            [10, 1]                   31
Total params: 401
Trainable params: 401
Non-trainable params: 0
Total mult-adds (M): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.01

[3]  데이터셋 클래스 설계 및 정의 <hr>
- 데이터셋: iris.csv
- 피쳐개수: 3개
- 타겟개수: 1개
- 클래스이름: IrisDataset
- 부모클래스: untils.data.Dataset
- 속성/필드: featureDF, TargetDF, n_rows, n_features
- 필수 메서드
    - __init__(self): 데이터셋 저장 및 전처리, 개발자가 필요한 속성 설정
    - __len__(slef): 데이터의 개수 반환
    - __getitem__(self, index): 특정 인덱스의 피쳐와 타겟 반환


In [26]:
class IrisDataset(Dataset):
    def __init__(self, featureDF, targetDF) -> None:
        super().__init__()
        self.featureDF= featureDF
        self.targetDF= targetDF
        self.n_rows= featureDF.shape[0]
        self.n_features= featureDF.shape[1]


    def __len__(self):
        return self.n_rows

    def __getitem__(self, index):
        # 텐서화 (전결합층=> 기본이 FloatTensor)
        featureTS= torch.FloatTensor(self.featureDF.iloc[index].values)
        targetTS= torch.FloatTensor(self.targetDF.iloc[index].values)
        return featureTS, targetTS

In [27]:
print(irisDF[irisDF.columns[-1]].shape)  ##인덱싱으로 추출하면 series, 
print(irisDF[irisDF.columns[-1:]].shape)  # 슬라이싱으로 하면 DF

(150,)
(150, 1)


In [28]:
# 데이터셋 인스턴스 생성
# shape은 항상 2D로!!!
irisDS=IrisDataset(irisDF[irisDF.columns[:-1]], irisDF[irisDF.columns[-1:]])
featureDF= irisDF[irisDF.columns[:-1]]
targetDF= irisDF[irisDF.columns[-1:]]

In [29]:
irisDS.n_rows, irisDS.n_features

(150, 3)

[4] 학습준비
- 학습 횟수: EPOCH (처음부터 끝까지 공부하는 단위)
- 배치 크기: BATCH_SIZE (한번에 학습할 데이터셋 양)
- 위치 지정: DEVICE 텐서 저장 및 실행 위치 (GPU, CPU)
- 학습률: 가중치와 절편 업데이트 시 경사하강법으로 업데이트하는 간격 (0.001~0.1=>HP), learning rate


In [30]:
# 학습 진행 관련 설정값

EPOCH= 20
BATCH_SIZE= 10  #버리는 데이터 없게 하려면 전체 데이터 수를 보고 조절
DEVICE= 'cuda' if torch.cuda.is_available() else 'cpu'
LR= 0.001

- 인스턴스/객체: 모델, 데이터셋, 최적화, (손실함수, 성능지표)

In [31]:
# 모델 인스턴스
model= IrisRegModel()
# 데이터셋 인스턴스
X_train, X_test, Y_train, Y_test= train_test_split(featureDF, targetDF, random_state=1)
X_train, X_val, Y_train, Y_val= train_test_split(X_train, Y_train, random_state=1)
print(X_train.shape, X_test.shape, X_val.shape)
irisDS= IrisDataset(X_train, Y_train) #검증용 데이터 클래스로 만들기 싫으면 나중에 텐서로 변환후 넣으면 됨
valDS=IrisDataset(X_val, Y_val)
testDS= IrisDataset(X_test, Y_test)
# 데이터로더 인스턴스 
irisDL= DataLoader(irisDS, batch_size= BATCH_SIZE)

(84, 3) (38, 3) (28, 3)


In [32]:
# 데이터로더 테스트
for feature, target in irisDL:
    print(feature.shape, target.shape)

torch.Size([10, 3]) torch.Size([10, 1])
torch.Size([10, 3]) torch.Size([10, 1])
torch.Size([10, 3]) torch.Size([10, 1])
torch.Size([10, 3]) torch.Size([10, 1])
torch.Size([10, 3]) torch.Size([10, 1])
torch.Size([10, 3]) torch.Size([10, 1])
torch.Size([10, 3]) torch.Size([10, 1])
torch.Size([10, 3]) torch.Size([10, 1])
torch.Size([4, 3]) torch.Size([4, 1])


In [33]:
# 최적화 인스턴스
# => w, b, model.parameters 전달
optimizer= optim.Adam(model.parameters(), lr=LR)


# 손실함수 인스턴스
# => MSE, MAE, RMSE
RegLoss=nn.MSELoss()


[5] 학습진행

>>> 모델 저장 준비 

In [34]:
# 학습된 모델 파라미터값 확인
model.state_dict()
# 모델 파라미터(가중치와 절편) 저장
import os
# 경로
SAVE_PATH= '../models/iris/'
# 이름
SAVE_FILE='model_train_wbs.pth'
# 모델 구조 및 파라미터 모두 저장 파일명
SAVE_MODEL='model_all.pth'

if not os.path.exists(SAVE_PATH):
    os.makedirs(SAVE_PATH)       #하위 폴더까지 생성

In [35]:
# 손실과 성능지표값 저장 => 학습의 효과 확인
LOSS_HISTORY, SCORE_HISTORY= [[],[]], [[],[]]


# 학습 모니터링/스케쥴링 설정 -> 두 HISTORY 활용
# 임계기준: 10번 (사용자지정)
BREAK_CNT=0
LIMIT=10



# 학습 모드로 모델 설정 (코딩 스타일에 따라 위치 상이)
model.train()

for epoch in range(EPOCH):                      #만약 검증을 한다면 Custom Dataset 만들어야함
    print(f'{epoch+1}/{EPOCH}')
    # 배치크기 만큼 데이터 로딩에서 학습 진행
    loss_total, score_total= 0,0
    for featureTS, targetTS in irisDL:
        # 학습 진행
        pre_y= model(featureTS)

        # 손실 계산
        loss= RegLoss(pre_y, targetTS)
        loss_total+=loss
        # 성능 평가
        score= R2Score()(pre_y, targetTS)
        score_total+= score

        # 최적화 진행
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    # 에포크당 검증기능
    # 검증 모드
    model.eval()
    with torch.no_grad():
        # 검증 데이터 셋
        val_featureTS=torch.FloatTensor(valDS.featureDF.values) #values -> array로 변경
        val_targetTS=torch.FloatTensor(valDS.targetDF.values)

        # 학습/추론론
        pre_val=model(val_featureTS)

        # 손실
        loss_val=RegLoss(pre_val, val_targetTS)

        # 평가
        score_val=R2Score()(pre_val, val_targetTS)



    # 손실값과 성능평가값 저장
    # 학습
    LOSS_HISTORY[0].append(loss_total/len(irisDL))
    SCORE_HISTORY[0].append(score_total/len(irisDL))
    # 검증
    LOSS_HISTORY[1].append(loss_val)
    SCORE_HISTORY[1].append(score_val)

    # 학습 진행 모니터링 (검증 데이터 개선이 되지 않았을때 누적 ->  평가, 손실 중 지표 하나 선택)
    if len(LOSS_HISTORY[0])>1:  #비교를 위해 데이터가 2개 이상 있어야함
        if LOSS_HISTORY[1][-1] >= LOSS_HISTORY[1][-2]: BREAK_CNT+=1   #개선이 되지 않았을때 카운딩

    # if len(SCORE_HISTORY[0])>1:  #비교를 위해 데이터가 2개 이상 있어야함
    #     if SCORE_HISTORY[1][-1] <= SCORE_HISTORY[1][-2]: BREAK_CNT+=1

    SAVE_FILE=f'model_train_{epoch}_{score_val:.4f}.pth'
    # 성능이 좋은 학습 가중치 저장
    if len(SCORE_HISTORY[1]) == 1: 
        #첫번째는 무조건 저장
        torch.save(model.state_dict(), SAVE_PATH+SAVE_FILE)  #파라미터
        torch.save(model, SAVE_PATH+SAVE_MODEL)                 #전체
    else:
        if SCORE_HISTORY[1][-1]> max(SCORE_HISTORY[1][:-1]): # 자신을 제외한 최대점수값과 비교
            torch.save(model.state_dict(), SAVE_PATH+SAVE_FILE) 
            torch.save(model, SAVE_PATH+SAVE_MODEL)  

    # 학습 중단 여부 설정
    if BREAK_CNT>LIMIT:
        print('성능 및 손실 개선이 없어서 학습 중단')
        break

1/20
2/20
3/20
4/20
5/20
6/20
7/20
8/20
9/20
10/20
11/20
12/20
13/20
14/20
15/20
16/20
17/20
18/20
19/20
20/20


In [36]:
# 테스트 데이터 성능과 비교

- 모델 저장 방법 <hr>
- 방법 1: 모델 파라미터만 저장
- 방법 2: 모델 설계 구조및 파라미터 모두 저장

In [37]:
# 모델 저장
torch.save(model.state_dict(), SAVE_PATH+SAVE_FILE)

In [38]:
# 모델 즉, 가중치와 절편 로딩
# [1] 가중치와 절편 객체로 로딩
#  [2] 모델의 state_dict 속성에 저장

# 읽기
wbTS= torch.load(SAVE_PATH+SAVE_FILE, weights_only=True)
print(type(wbTS))

<class 'collections.OrderedDict'>


In [39]:
# 모델 인스턴스에 저장
model.load_state_dict(wbTS)
model2=IrisRegModel()
model2.load_state_dict(wbTS)

<All keys matched successfully>